In [502]:
print("ok")

ok


In [503]:
%pwd

'd:\\'

In [504]:
import os

os.chdir("../")
%pwd

'd:\\'

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from typing import List
from langchain.schema import Document

from langchain_community.embeddings import HuggingFaceEmbeddings
import re
import unicodedata

from langchain_community.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [506]:
def load_pdf_files(data):

    loader = DirectoryLoader(  #Created to load all PDF files from a directory
        data,
        glob = "*.pdf",
        loader_cls=PyPDFLoader,
    )

    documents = loader.load()
    return documents

extracted_data = load_pdf_files("D:\Advanced-QNA-RAG\data")


In [507]:
extracted_data[33:34]  #Displaying first two loaded documents

[Document(metadata={'source': 'D:\\Advanced-QNA-RAG\\data\\Three Thousand Stitches by Sudha Murthy.pdf', 'page': 33}, page_content='others\tsay\tit\tcame\tfrom\tMexico.\tAccording\tto\ta\ttheory,\tblack\tpepper\twas\tthe\ningredient\ttraditionally\tused\tin\tIndia\tto\tmake\tour\tfood\thot\tand\tspicy.\tSome\nscholars\tbelieve\tthat\tthe\tsole\tgoal\tof\tthe\tEast\tIndia\tCompany\twas\tto\tacquire\ta\nmonopoly\tover\tIndia’s\tpepper\ttrade,\n\twhich\tlater\tended\tin\tIndia’s\tcolonization.\nBut\twhen\twe\tbegan\tusing\tchillies,\twe\tfound\tthat\tit\ttasted\tbetter\tthan\tblack\npepper.\tTo\tgive\tyou\tan\texample,\twe\trefer\tto\tblack\tpepper\tas\t\nkalu\tmenasu\n\tin\nKannada.\tWe\tgave\ta\tsimilar\tname\tto\tthe\tchilli\tand\tcalled\tit\t\nmenasin\tkai\n.\tIn\nHindi,\tit\tis\tfrequently\treferred\tto\tas\t\nmirchi\n.\tIn\tthe\twar\tbetween\tblack\tpepper\tand\nchilli,\tthe\tformer\tlost\tand\tchilli\testablished\n\titself\tas\tthe\tnew\tprince\tand\tcontinues\nto\trule\tthe\tIndia

In [508]:
print(type(extracted_data))


<class 'list'>


In [538]:
# [i for i in extracted_data[34].metadata.items()]  #Displaying metadata of first document

[i for i in extracted_data[2]]  #Displaying metadata of first document

[('id', None),
 ('metadata',
  {'source': 'D:\\Advanced-QNA-RAG\\data\\Three Thousand Stitches by Sudha Murthy.pdf',
   'page': 2}),
 ('page_content',
  'SUDHA\tMURTY\n\t\nTHREE\tTHOUSAND\tSTITCHES\nOrdinary\tPeople,\tExtraordinary\tLives\nPENGUIN\tBOOKS'),
 ('type', 'Document')]

In [510]:

# data = [
#     ('id', None),
#     ('metadata',
#      {'producer': 'calibre 3.42.0 [https://calibre-ebook.com]',
#       'creator': 'calibre 3.42.0 [https://calibre-ebook.com]',
#       'creationdate': '2019-07-12T08:02:34+00:00',
#       'author': 'Sudha Murty',
#       'title': 'Three Thousand Stitches - PDFDrive.com',
#       'source': 'data\\Three Thousand Stitches by Sudha Murthy.pdf',
#       'total_pages': 128,
#       'page': 3,
#       'page_label': '4'}),
#     ('page_content', '...'),
#     ('type', 'Document')
# ]

# # Extract producer
# producer = next((value['producer'] for key, value in data if key == 'metadata'), None)
# print(producer)

In [ ]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [512]:
minimal_docs = filter_to_minimal_docs(extracted_data)
minimal_docs[0]

Document(metadata={'source': 'D:\\Advanced-QNA-RAG\\data\\Three Thousand Stitches by Sudha Murthy.pdf'}, page_content='')

In [513]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(  # defined to split text into smaller chunks
        chunk_size=300,
        chunk_overlap=50,
        separators=["\n\n", "\n", ". ", "! ", "? ", "; ", ", ", " "],
        length_function=len
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk


In [514]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 1044


In [ ]:
# import re
# import unicodedata

# def clean_text(texts_chunk):
#     text = unicodedata.normalize("NFKD", texts_chunk.page_content)
#     text = text.lower()
#     text = re.sub(r"\s+", " ", text)  # Collapse whitespace
#     texts_chunk = re.sub(r"[^\w\s.,:;!?()-]", "", text)  # Remove junk characters
#     return text.strip()

In [516]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 1044


In [517]:
texts_chunk[0]  #Displaying first chunk

Document(metadata={'source': 'D:\\Advanced-QNA-RAG\\data\\Three Thousand Stitches by Sudha Murthy.pdf'}, page_content='SUDHA\tMURTY\n\t\nTHREE\tTHOUSAND\tSTITCHES\nOrdinary\tPeople,\tExtraordinary\tLives\nPENGUIN\tBOOKS')

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import re
import unicodedata

def download_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

# Test it
embeddings = download_embeddings()
print("Embeddings loaded successfully!")

Embeddings loaded successfully!


In [519]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [520]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get API keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")  # ✅ This line was missing

# Set them in environment (optional, if needed by other libraries)
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [521]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

pc

In [ ]:
from pinecone import Pinecone, ServerlessSpec

index_name = "advanced-qna-rag"

# Check if index exists by listing all indexes
existing_indexes = pc.list_indexes().names()

if index_name not in existing_indexes:
    print(f"Creating index: {index_name}")
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension of the embeddings
        metric="cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"Index '{index_name}' created successfully!")
else:
    print(f"Index '{index_name}' already exists.")

# Connect to the index
index = pc.Index(index_name)
print(f"Connected to index: {index_name}")

Index 'chatbot' already exists.
Connected to index: chatbot


In [ ]:
from langchain_community.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [524]:
from langchain_pinecone import PineconeVectorStore
print("PineconeVectorStore imported successfully!")

PineconeVectorStore imported successfully!


In [525]:
from langchain_pinecone import PineconeVectorStore

# Assuming texts_chunk and embedding are defined from earlier cells
docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embeddings,  # Note: Your notebook uses 'embeddings' in cell 31, which should be 'embedding'
    index_name=index_name
)
print("Vector store created successfully!")

Vector store created successfully!


In [526]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":9})

In [527]:
retrieved_docs = retriever.invoke("Who is sudha murthy?")
# retrieved_docs

In [528]:
from langchain_groq import ChatGroq
import os

# Update your model to a working one
chatModel = ChatGroq(
    model="groq/compound",  # Working model
    # model="openai/gpt-oss-120b",  # Working model,
    temperature=0.2,
    groq_api_key=os.environ.get("GROQ_API_KEY")
)

print(f"Using model: {chatModel.model_name}")

Using model: groq/compound


In [529]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
#chat model has two input system & User prompt

system_prompt = (
    "You are the best assistant for answering questions based on the provided context. "
    "Use the context below to answer accurately and factually. "
    "If you don't know the answer, say clearly You don't know. "
    "Limit your response to two sentences.\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
#Create a Chain

question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [532]:
response = rag_chain.invoke({"input": "What specific academic challenges and forms of harassment did the narrator face from her classmates in engineering college??"})
print(response["answer"])

The narrator was repeatedly told that women didn’t belong in engineering—she was expected to study medicine or science with other female students, and her presence in an engineering program was dismissed as “like expecting pigs to fly.” Her classmates expressed that bias through constant sarcasm, pranks, pitying remarks (e.g., comparing her to Goddess Parvati and questioning why she chose engineering) and demeaning comments that questioned her motives and ability.


In [533]:
response = rag_chain.invoke({"input": "How did the narrator's psychological coping strategies and academic performance interact to help her overcome the hostile environment??"})
print(response["answer"])

College taught her resilience, flexibility, and collaborative skills—psychological coping tools that let her transform fear into confidence—while her academic success reinforced her self‑identity as a capable engineer; together, these strategies enabled her to confront hostility, build supportive relationships, and ultimately thrive despite the hostile environment.


In [534]:
response = rag_chain.invoke({"input":"What does the narrator's journey from being an alien to gaining respect and making friends reveal about the transformative power of competence and persistence in breaking gender barriers?"})
print(response["answer"])

The narrator’s transformation—from an “alien” outsider to a respected peer—demonstrates that demonstrable competence and relentless persistence can overturn gendered expectations, earning admiration and friendship even in hostile environments. By consistently proving skill and resilience, the narrator dismantles stereotypes, showing that sustained excellence is a powerful catalyst for breaking gender barriers.


In [535]:
response = rag_chain.invoke({"input": "Considering both the societal context described and the narrator's personal transformation, what does this passage suggest about the relationship between individual excellence and systemic change in challenging gender stereotypes in male-dominated fields?"})
print(response["answer"])

The passage illustrates that individual excellence—whether it is the narrator’s own academic and professional achievements, his father’s unusually liberal support for his daughters’ education, or the doctor‑teacher’s reputation for caring for women—acts as a catalyst that both exposes and begins to erode entrenched gender stereotypes in male‑dominated arenas. At the same time, the narrative makes clear that personal success alone cannot overturn systemic bias; it must be reinforced by concrete institutional changes (such as the provision of women’s toilets on campus), collective advocacy, and supportive networks that validate and amplify women’s aspirations, thereby turning isolated instances of excellence into a broader, sustainable shift in cultural expectations and structural opportunities.


In [ ]:
# https://chat.deepseek.com/share/8fp97r8nzc6i11es4k

In [ ]:
# !pip install python-dotenv
